In [2]:
import pandas as pd
from pandas.core.computation.check import NUMEXPR_INSTALLED
import numpy as np 
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re

In [3]:
final_data = []
year = 2015
weeks = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

for week in weeks:
    url = f'https://www.espn.com/nfl/scoreboard/_/week/{week}/year/{year}/seasontype/2'
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')

    week_data = []
    for board in soup.select('.ScoreboardScoreCell'):
        title = board.find_previous(class_='Card__Header__Title').text
        teams = [t.text for t in board.select('.ScoreCell__TeamName')]
        scores = [s.text for s in board.select('.ScoreCell__Score')] or ['-', '-']

        week_data.append((week, title, teams[0], scores[0], teams[1], scores[1]))

    final_data.extend(week_data)

df_scores_2015 = pd.DataFrame(final_data, columns=['Week', 'Date', 'Team 1', 'Score 1', 'Team 2', 'Score 2'])
df_scores_2015 = df_scores_2015.rename(columns={'Team 1': 'Away Team', 'Score 1': 'Away Score', 'Team 2': 'Home Team', 'Score 2':'Home Score'})
df_scores_2015['Final Score'] = df_scores_2015['Away Score'] + ' - ' + df_scores_2015['Home Score']
#print(df.to_markdown(index=False))
# Replace missing scores with NaN and dtype convert object to int 
df_scores_2015['Away Score'] = df_scores_2015['Away Score'].replace('-', np.nan)
df_scores_2015['Home Score'] = df_scores_2015['Home Score'].replace('-', np.nan)
df_scores_2015['Away Score'] = pd.to_numeric(df_scores_2015['Away Score'])
df_scores_2015['Home Score'] =  pd.to_numeric(df_scores_2015['Home Score'])

# Calculate total points
df_scores_2015['Total Points'] = df_scores_2015['Away Score'] + df_scores_2015['Home Score']

# Calculate winner
df_scores_2015['Winner'] = df_scores_2015.apply(lambda row: row['Away Team'] if row['Away Score'] > row['Home Score'] else row['Home Team'], axis=1)

# Calculate score differential
df_scores_2015['Away Score Differential'] = df_scores_2015['Home Score'] - df_scores_2015['Away Score']
df_scores_2015['Home Score Differential'] = df_scores_2015['Away Score'] - df_scores_2015['Home Score']
df_scores_2015['Matchup'] = df_scores_2015['Away Team'] + ' @ ' + df_scores_2015['Home Team']
df_scores_2015['Date'] = pd.to_datetime(df_scores_2015['Date'], format='%A, %B %d, %Y').dt.strftime('%Y-%m-%d')
# Use a lambda function to create the 'Home_Away_Winner' column
df_scores_2015['Home_Away_Winner'] = df_scores_2015.apply(lambda row: 'Home' 
                                                          if row['Winner'] == row['Home Team'] 
                                                          else 'Away' if row['Winner'] == row['Away Team'] else 'No Winner', axis=1)
df_scores_2015.tail()

,Week,Date,Away Team,Away Score,Home Team,Home Score,Final Score,Total Points,Winner,Away Score Differential,Home Score Differential,Matchup,Home_Away_Winner
251,17,2016-01-03,Raiders,17,Chiefs,23,17 - 23,40,Chiefs,6,-6,Raiders @ Chiefs,Home
252,17,2016-01-03,Seahawks,36,Cardinals,6,36 - 6,42,Seahawks,-30,30,Seahawks @ Cardinals,Away
253,17,2016-01-03,Rams,16,49ers,19,16 - 19,35,49ers,3,-3,Rams @ 49ers,Home
254,17,2016-01-03,Buccaneers,10,Panthers,38,10 - 38,48,Panthers,28,-28,Buccaneers @ Panthers,Home
255,17,2016-01-03,Vikings,20,Packers,13,20 - 13,33,Vikings,-7,7,Vikings @ Packers,Away


In [4]:
#import consensus data for spreads 
game_dates = list(df_scores_2015['Date'].unique())
uri_list = []
uri = "https://contests.covers.com/consensus/topconsensus/nfl/overall/"

for date in game_dates:
    uri_list.append(uri + date)

spreads = []
for item in uri_list:
    try:
        res = requests.get(item)
        soup = BeautifulSoup(res.content, 'lxml')
        table = soup.find_all('table')[0]
        temp_df = pd.read_html(str(table))[0]  # Obtain the first table from the list
        temp_df['Date'] = re.search(r'\d{4}-\d{2}-\d{2}', item).group()  # Add the corresponding date column

        spreads.append(temp_df)
    except IndexError:
        print(f"IndexError occurred for URL: {item}")
        continue
    except Exception as e:
        print(f"An error occurred for URL: {item}")
        print(f"Error details: {str(e)}")
        continue

df_spreads_2015 = pd.concat(spreads, ignore_index=True)

In [5]:
#clean matchup column 
df_spreads_2015['Matchup'] = df_spreads_2015['Matchup'].str[4:].str.upper().str.replace("  ",' @ ')
# clean add home and away column 
df_spreads_2015[['Away Team', 'Home Team']] = df_spreads_2015['Matchup'].str.split(' @ ', expand=True)
# Remove leading/trailing whitespaces
df_spreads_2015['Away Team'] = df_spreads_2015['Away Team'].str.strip()
df_spreads_2015['Home Team'] = df_spreads_2015['Home Team'].str.strip()
#split to create away and home spread cols 
df_spreads_2015[['Away Spread', 'Home Spread']] = df_spreads_2015['Sides'].str.split(expand=True)
#split to create away and home consensus cols 
df_spreads_2015[['Away Consensus', 'Home Consensus']] = df_spreads_2015['Consensus'].str.split(expand=True)
# convert to dtype = float for consensus cols 
df_spreads_2015['Away Consensus'] =  df_spreads_2015['Away Consensus'].str.rstrip('%').astype(int)/100
df_spreads_2015['Home Consensus'] = df_spreads_2015['Home Consensus'].str.rstrip('%').astype(int)/100
#create consensus favorite vs. underdog cols 
df_spreads_2015['Con. Spread Favorite'] = df_spreads_2015.apply(lambda x: x['Away Team'] 
                                                                if x['Away Consensus'] > x['Home Consensus'] else x['Home Team'], axis=1)
df_spreads_2015['Con. Spread Underdog'] = df_spreads_2015.apply(lambda x: x['Away Team'] 
                                                                if x['Away Consensus'] < x['Home Consensus'] else x['Home Team'], axis=1)
# Define the dictionary mapping for replacements
replacement_dict = {'LV': 'Raiders',
                    'KC': 'Chiefs',
                    'NE': 'Patriots',
                    'GB': 'Packers',
                    'NO': 'Saints',
                    'BAL': 'Ravens',
                    'SF': '49ers',
                    'NYG': 'Giants',
                    'JAC': 'Jaguars',
                    'PIT': 'Steelers',
                    'TB': 'Buccaneers',
                    'CLE': 'Browns',
                    'IND': 'Colts',
                    'PHI': 'Eagles',
                    'ARI': 'Cardinals',
                    'WAS': 'Commanders',
                    'ATL': 'Falcons',
                    'NYJ': 'Jets',
                    'SEA': 'Seahawks',
                    'MIA': 'Dolphins',
                    'HOU': 'Texans',
                    'CAR': 'Panthers',
                    'CHI': 'Bears',
                    'CIN': 'Bengals',
                    'LAR': 'Rams',
                    'BUF': 'Bills',
                    'DET': 'Lions',
                    'DEN': 'Broncos',
                    'TEN': 'Titans',
                    'MIN': 'Vikings',
                    'LAC': 'Chargers',
                    'DAL': 'Cowboys'}
# Replace the values in the DataFrame column using map()
df_spreads_2015['Away Team'] = df_spreads_2015['Away Team'].map(replacement_dict)
df_spreads_2015['Home Team'] = df_spreads_2015['Home Team'].map(replacement_dict)
df_spreads_2015['Con. Spread Favorite'] = df_spreads_2015['Con. Spread Favorite'].map(replacement_dict)
df_spreads_2015['Con. Spread Underdog'] = df_spreads_2015['Con. Spread Underdog'].map(replacement_dict)
df_spreads_2015['Matchup'] = df_spreads_2015['Away Team'] + ' @ ' + df_spreads_2015['Home Team']
df_spreads_2015.tail()

,Matchup,Date,Consensus,Sides,Picks,Indepth,Away Team,Home Team,Away Spread,Home Spread,Away Consensus,Home Consensus,Con. Spread Favorite,Con. Spread Underdog
251,Seahawks @ Cardinals,2016-01-03,54% 46%,+6 -6,1140 967,Details,Seahawks,Cardinals,+6,-6,0.54,0.46,Seahawks,Cardinals
252,Raiders @ Chiefs,2016-01-03,54% 46%,+6.5 -6.5,1054 906,Details,Raiders,Chiefs,+6.5,-6.5,0.54,0.46,Raiders,Chiefs
253,Patriots @ Dolphins,2016-01-03,54% 46%,-10 +10,1014 880,Details,Patriots,Dolphins,-10,+10,0.54,0.46,Patriots,Dolphins
254,Ravens @ Bengals,2016-01-03,53% 47%,+10 -10,1023 919,Details,Ravens,Bengals,+10,-10,0.53,0.47,Ravens,Bengals
255,Saints @ Falcons,2016-01-03,49% 51%,+6 -6,969 1002,Details,Saints,Falcons,+6,-6,0.49,0.51,Falcons,Saints


In [6]:
#import consensus data for spreads 
game_dates = list(df_scores_2015['Date'].unique())
uri_list = []
uri = "https://contests.covers.com/consensus/topoverunderconsensus/nfl/overall/"

for date in game_dates:
    uri_list.append(uri + date)

totals = []
for item in uri_list:
    try:
        res = requests.get(item)
        soup = BeautifulSoup(res.content, 'lxml')
        table = soup.find_all('table')[0]
        temp_df = pd.read_html(str(table))[0]  # Obtain the first table from the list
        temp_df['Date'] = re.search(r'\d{4}-\d{2}-\d{2}', item).group()  # Add the corresponding date column

        totals.append(temp_df)
    except IndexError:
        print(f"IndexError occurred for URL: {item}")
        continue
    except Exception as e:
        print(f"An error occurred for URL: {item}")
        print(f"Error details: {str(e)}")
        continue

df_totals_2015 = pd.concat(totals, ignore_index=True)
df_totals_2015.tail()

,Matchup,Date,Consensus,Total,Picks,Indepth
251,NFL Lar Sf,2016-01-03,53 % Over 47 % Under,39.0,749 673,Details
252,NFL Nyj Buf,2016-01-03,52 % Over 48 % Under,41.0,731 671,Details
253,NFL Ten Ind,2016-01-03,52 % Under 48 % Over,39.0,291 268,Details
254,NFL Jac Hou,2016-01-03,52 % Over 48 % Under,45.0,630 589,Details
255,NFL Pit Cle,2016-01-03,51 % Over 49 % Under,47.0,717 698,Details


In [7]:
#clean matchup column 
df_totals_2015['Matchup'] = df_totals_2015['Matchup'].str[4:].str.upper().str.replace("  ",' @ ')
# clean add home and away column 
df_totals_2015[['Away Team', 'Home Team']] = df_totals_2015['Matchup'].str.split(' @ ', expand=True)
# Remove leading/trailing whitespaces
df_totals_2015['Away Team'] = df_totals_2015['Away Team'].str.strip()
df_totals_2015['Home Team'] = df_totals_2015['Home Team'].str.strip()
# create consensus columns 
df_totals_2015['Over Consensus'] = df_totals_2015['Consensus'].str.extract(r'(\d+) % Over').astype(int)/100
df_totals_2015['Under Consensus'] = df_totals_2015['Consensus'].str.extract(r'(\d+) % Under').astype(int)/100
#create consensus favorites vs underdogs 
df_totals_2015['Con. Total Favorite'] = df_totals_2015[['Over Consensus', 'Under Consensus']].apply(lambda x: 'Over' 
                                                                                                    if x['Over Consensus'] > x['Under Consensus'] else 'Under', axis=1)
df_totals_2015['Con. Total Underdog'] = df_totals_2015[['Over Consensus', 'Under Consensus']].apply(lambda x: 'Over' 
                                                                                                    if x['Over Consensus'] < x['Under Consensus'] else 'Under', axis=1)
# Replace the values in the DataFrame column using map() from our previously defined replacement_dict
df_totals_2015['Away Team'] = df_totals_2015['Away Team'].map(replacement_dict)
df_totals_2015['Home Team'] = df_totals_2015['Home Team'].map(replacement_dict)
df_totals_2015['Matchup'] = df_totals_2015['Away Team'] + ' @ ' + df_totals_2015['Home Team']
df_totals_2015.tail()

,Matchup,Date,Consensus,Total,Picks,Indepth,Away Team,Home Team,Over Consensus,Under Consensus,Con. Total Favorite,Con. Total Underdog
251,Rams @ 49ers,2016-01-03,53 % Over 47 % Under,39.0,749 673,Details,Rams,49ers,0.53,0.47,Over,Under
252,Jets @ Bills,2016-01-03,52 % Over 48 % Under,41.0,731 671,Details,Jets,Bills,0.52,0.48,Over,Under
253,Titans @ Colts,2016-01-03,52 % Under 48 % Over,39.0,291 268,Details,Titans,Colts,0.48,0.52,Under,Over
254,Jaguars @ Texans,2016-01-03,52 % Over 48 % Under,45.0,630 589,Details,Jaguars,Texans,0.52,0.48,Over,Under
255,Steelers @ Browns,2016-01-03,51 % Over 49 % Under,47.0,717 698,Details,Steelers,Browns,0.51,0.49,Over,Under


In [8]:
print(df_scores_2015.columns)
print(df_spreads_2015.columns)
print(df_totals_2015.columns)

Index(['Week', 'Date', 'Away Team', 'Away Score', 'Home Team', 'Home Score',
       'Final Score', 'Total Points', 'Winner', 'Away Score Differential',
       'Home Score Differential', 'Matchup', 'Home_Away_Winner'],
      dtype='object')
Index(['Matchup', 'Date', 'Consensus', 'Sides', 'Picks', 'Indepth',
       'Away Team', 'Home Team', 'Away Spread', 'Home Spread',
       'Away Consensus', 'Home Consensus', 'Con. Spread Favorite',
       'Con. Spread Underdog'],
      dtype='object')
Index(['Matchup', 'Date', 'Consensus', 'Total', 'Picks', 'Indepth',
       'Away Team', 'Home Team', 'Over Consensus', 'Under Consensus',
       'Con. Total Favorite', 'Con. Total Underdog'],
      dtype='object')


In [9]:
# pd.merge(df1, df2, on=['Key_Column_1', 'Key_Column_2'], how='inner')
combined_df = pd.merge(df_scores_2015[['Week', 'Date', 'Matchup', 'Final Score', 'Total Points', 
                                       'Winner', 'Home_Away_Winner', 'Away Team', 'Away Score', 
                                       'Away Score Differential', 'Home Team', 'Home Score', 'Home Score Differential']], 
                       df_spreads_2015[['Matchup', 'Date', 'Away Team','Away Spread', 'Home Team', 
                                        'Home Spread', 'Away Consensus', 'Home Consensus', 
                                        'Con. Spread Favorite','Con. Spread Underdog']],
                       on=['Matchup', 'Date', 'Away Team', 'Home Team'], 
                       how='inner')

In [10]:
final_df_nfl_2015 = pd.merge(combined_df[['Week', 'Date', 'Matchup', 'Final Score', 'Total Points', 'Winner',
                                           'Home_Away_Winner', 'Away Team', 'Away Score',
                                           'Away Score Differential', 'Home Team', 'Home Score',
                                           'Home Score Differential', 'Away Spread', 'Home Spread',
                                           'Away Consensus', 'Home Consensus', 'Con. Spread Favorite',
                                           'Con. Spread Underdog']],
                            df_totals_2015[['Matchup', 'Date', 'Away Team', 'Home Team', 'Total',
                                            'Over Consensus', 'Under Consensus',
                                            'Con. Total Favorite', 'Con. Total Underdog']],
                             on=['Matchup', 'Date', 'Away Team', 'Home Team'], 
                             how='inner')

final_df_nfl_2015['Win Marginal'] = np.abs(final_df_nfl_2015['Away Score'] - final_df_nfl_2015['Home Score'])

In [11]:
# 1) did away vs home team cover? -- Done 
# Convert 'Away Spread' and 'Home Spread' columns to numeric values
final_df_nfl_2015['Away Spread'] = pd.to_numeric(final_df_nfl_2015['Away Spread'])
final_df_nfl_2015['Home Spread'] = pd.to_numeric(final_df_nfl_2015['Home Spread'])

final_df_nfl_2015['Spread Covering Team'] = final_df_nfl_2015.apply(lambda row: row['Home Team'] 
                                                              if row['Home Score'] + row['Home Spread'] >= row['Away Score'] 
                                                              else row['Away Team'], axis=1)
final_df_nfl_2015[['Matchup', 'Final Score','Home Team', 'Home Spread', 'Home Score', 'Home Spread','Away Team', 'Away Spread', 'Spread Covering Team']].head()

,Matchup,Final Score,Home Team,Home Spread,Home Score,Home Spread,Away Team,Away Spread,Spread Covering Team
0,Steelers @ Patriots,21 - 28,Patriots,0.0,28,0.0,Steelers,0.0,Patriots
1,Colts @ Bills,14 - 27,Bills,0.0,27,0.0,Colts,0.0,Bills
2,Packers @ Bears,31 - 23,Bears,0.0,23,0.0,Packers,0.0,Packers
3,Seahawks @ Rams,31 - 34,Rams,0.0,34,0.0,Seahawks,0.0,Rams
4,Browns @ Jets,10 - 31,Jets,0.0,31,0.0,Browns,0.0,Jets


In [12]:
# 2) Did over/under hit? -- Done

# Create "Totals Covered" column
final_df_nfl_2015['Totals Covering Line'] = final_df_nfl_2015.apply(lambda row: 'Over' 
                                          if row['Total Points'] > row['Total'] 
                                          else ('Under' if row['Total Points'] < row['Total'] else 'Push'), axis=1)

# Print the updated DataFrame
final_df_nfl_2015[['Matchup', 'Total Points', 'Total', 'Totals Covering Line']].head()

,Matchup,Total Points,Total,Totals Covering Line
0,Steelers @ Patriots,49,0.0,Over
1,Colts @ Bills,41,0.0,Over
2,Packers @ Bears,54,0.0,Over
3,Seahawks @ Rams,65,0.0,Over
4,Browns @ Jets,41,0.0,Over


In [13]:
# 3) Did consensus fav vs underdog win? [Spread] -- Done

# Create "Consensus Spread Winner" column
final_df_nfl_2015['Consensus Spread Winner'] = final_df_nfl_2015.apply(lambda row: 'Majority' 
                                            if row['Spread Covering Team'] in [row['Away Team'], row['Home Team']] and
                                                          ((row['Spread Covering Team'] == row['Away Team'] and row['Away Consensus'] > 0.5) or
                                                           (row['Spread Covering Team'] == row['Home Team'] and row['Home Consensus'] > 0.5))
                                            else 'Minority' if row['Spread Covering Team'] in [row['Away Team'], row['Home Team']] and
                                                          ((row['Spread Covering Team'] == row['Away Team'] and row['Away Consensus'] < 0.5) or
                                                           (row['Spread Covering Team'] == row['Home Team'] and row['Home Consensus'] < 0.5))
                                            else 'Push' if row['Spread Covering Team'] in [row['Away Team'], row['Home Team']] and
                                                          ((row['Spread Covering Team'] == row['Away Team'] and row['Away Consensus'] == 0.5) or
                                                           (row['Spread Covering Team'] == row['Home Team'] and row['Home Consensus'] == 0.5))
                                            else '', axis=1)

# Print the updated DataFrame
final_df_nfl_2015[['Away Team', 'Home Team', 'Away Consensus', 'Home Consensus', 'Consensus Spread Winner']].head()

,Away Team,Home Team,Away Consensus,Home Consensus,Consensus Spread Winner
0,Steelers,Patriots,0.48,0.52,Majority
1,Colts,Bills,0.63,0.37,Minority
2,Packers,Bears,0.69,0.31,Majority
3,Seahawks,Rams,0.58,0.42,Minority
4,Browns,Jets,0.39,0.61,Majority


In [14]:
# 4) Did consensus fav vs underdog win? [Totals] -- Done
# Create "Total Consensus Winner" column
final_df_nfl_2015['Total Consensus Winner'] = final_df_nfl_2015.apply(lambda row: 'Majority' 
                                                  if ((row['Totals Covering Line'] == 'Under' and row['Under Consensus'] > 0.5) or
                                                                   (row['Totals Covering Line'] == 'Over' and row['Over Consensus'] > 0.5))
                                                  else 'Minority' if ((row['Totals Covering Line'] == 'Under' and row['Under Consensus'] < 0.5) or
                                                                   (row['Totals Covering Line'] == 'Over' and row['Over Consensus'] < 0.5))
                                                  else 'Push' if (row['Under Consensus'] == 0.5) or (row['Over Consensus'] == 0.5)
                                                  else '', axis=1)

# Print the updated DataFrame
#final_df_nfl_2015[['Matchup', 'Total Points', 'Total', 'Totals Covering Line', 'Over Consensus', 'Away Consensus', 'Total Consensus Winner']]

In [15]:
final_df_nfl_2015.columns

Index(['Week', 'Date', 'Matchup', 'Final Score', 'Total Points', 'Winner',
       'Home_Away_Winner', 'Away Team', 'Away Score',
       'Away Score Differential', 'Home Team', 'Home Score',
       'Home Score Differential', 'Away Spread', 'Home Spread',
       'Away Consensus', 'Home Consensus', 'Con. Spread Favorite',
       'Con. Spread Underdog', 'Total', 'Over Consensus', 'Under Consensus',
       'Con. Total Favorite', 'Con. Total Underdog', 'Win Marginal',
       'Spread Covering Team', 'Totals Covering Line',
       'Consensus Spread Winner', 'Total Consensus Winner'],
      dtype='object')

In [16]:
final_df_nfl_2015

,Week,Date,Matchup,Final Score,Total Points,Winner,Home_Away_Winner,Away Team,Away Score,Away Score Differential,...,Total,Over Consensus,Under Consensus,Con. Total Favorite,Con. Total Underdog,Win Marginal,Spread Covering Team,Totals Covering Line,Consensus Spread Winner,Total Consensus Winner
0,1,2015-09-10,Steelers @ Patriots,21 - 28,49,Patriots,Home,Steelers,21,7,...,0.0,0.55,0.45,Over,Under,7,Patriots,Over,Majority,Majority
1,1,2015-09-13,Colts @ Bills,14 - 27,41,Bills,Home,Colts,14,13,...,0.0,0.53,0.47,Over,Under,13,Bills,Over,Minority,Majority
2,1,2015-09-13,Packers @ Bears,31 - 23,54,Packers,Away,Packers,31,-8,...,0.0,0.57,0.43,Over,Under,8,Packers,Over,Majority,Majority
3,1,2015-09-13,Seahawks @ Rams,31 - 34,65,Rams,Home,Seahawks,31,3,...,0.0,0.41,0.59,Under,Over,3,Rams,Over,Minority,Minority
4,1,2015-09-13,Browns @ Jets,10 - 31,41,Jets,Home,Browns,10,21,...,0.0,0.31,0.69,Under,Over,21,Jets,Over,Majority,Minority
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,17,2016-01-03,Raiders @ Chiefs,17 - 23,40,Chiefs,Home,Raiders,17,6,...,43.5,0.65,0.35,Over,Under,6,Raiders,Under,Majority,Minority
236,17,2016-01-03,Seahawks @ Cardinals,36 - 6,42,Seahawks,Away,Seahawks,36,-30,...,46.5,0.47,0.53,Under,Over,30,Seahawks,Under,Majority,Majority
237,17,2016-01-03,Rams @ 49ers,16 - 19,35,49ers,Home,Rams,16,3,...,39.0,0.53,0.47,Over,Under,3,49ers,Under,Minority,Minority
238,17,2016-01-03,Buccaneers @ Panthers,10 - 38,48,Panthers,Home,Buccaneers,10,28,...,44.0,0.61,0.39,Over,Under,28,Panthers,Over,Minority,Majority


In [17]:
final_df_nfl_2015.to_csv('final_df_nfl_2015.csv')